In [3]:
import math
import sys


class Rect:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2

    def perimeter(self):
        return 2 * (abs(self.x2 - self.x1) + abs(self.y2 - self.y1))

    def is_overlap(self, rect):
        if self.y1 > rect.y2 or self.y2 < rect.y1 or self.x1 > rect.x2 or self.x2 < rect.y1:
            return False
        return True

    def contain_rect(self, rect):
        return self.x1 < rect.x1 and self.y1 < rect.y1 and self.x2 > rect.x2 and self.y2 > rect.y2

    def has_point(self, point):
        return self.x1 <= point.x <= self.x2 and self.y1 <= point.y <= self.y2

    def __str__(self):
        return "Rect: ({}, {}), ({}, {})".format(self.x1, self.y1, self.x2, self.y2)


class Point:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y

    def __str__(self):
        return "Point #{}: ({}, {})".format(self.id, self.x, self.y)


def sequential_query(points, query):
    result = 0
    for point in points:
        if query.x1 <= point.x <= query.x2 and query.y1 <= point.y <= query.y2:
            result = result + 1
    return result


class Node(object):
    def __init__(self, d, n):
        self.d = d
        self.n = n
        self.id = 0
        # for internal nodes
        self.child_nodes = []
        # for leaf nodespyth
        self.data_points = []
        self.parent_node = None
        self.MBR = Rect(-1, -1, -1, -1)

    def add_point(self, point):
        # update in the right position to keep the list ordered
        self.add_points([point])
        pass

    def add_points(self, points):
        self.data_points += points
        # update MBR
        self.update_MBR()
        pass

    def perimeter_increase_with_point(self, point):
        x1 = point.x if point.x < self.MBR.x1 else self.MBR.x1
        y1 = point.y if point.y < self.MBR.y1 else self.MBR.y1
        x2 = point.x if point.x > self.MBR.x2 else self.MBR.x2
        y2 = point.y if point.y > self.MBR.y2 else self.MBR.y2
        return Rect(x1, y1, x2, y2).perimeter() - self.perimeter()

    def perimeter(self):
        # only calculate the half perimeter here
        return self.MBR.perimeter()

    def is_underflow(self):
        return (self.is_leaf() and len(self.data_points) < math.ceil(self.n / 2)) or \
               (not self.is_leaf() and len(self.child_nodes) < math.ceil(self.d / 2))

    def is_overflow(self):
        return (self.is_leaf() and len(self.data_points) > self.n) or \
               (not self.is_leaf() and len(self.child_nodes) > self.d)

    def is_root(self):
        return self.parent_node is None

    def is_leaf(self):
        return len(self.child_nodes) == 0

    def add_child_node(self, node):
        self.add_child_nodes([node])
        pass

    def add_child_nodes(self, nodes):
        for node in nodes:
            node.parent_node = self
            self.child_nodes.append(node)
        self.update_MBR()
        pass

    def update_MBR(self):
        if self.is_leaf():
            self.MBR.x1 = min([point.x for point in self.data_points])
            self.MBR.x2 = max([point.x for point in self.data_points])
            self.MBR.y1 = min([point.y for point in self.data_points])
            self.MBR.y2 = max([point.y for point in self.data_points])
        else:
            self.MBR.x1 = min([child.MBR.x1 for child in self.child_nodes])
            self.MBR.x2 = max([child.MBR.x2 for child in self.child_nodes])
            self.MBR.y1 = min([child.MBR.y1 for child in self.child_nodes])
            self.MBR.y2 = max([child.MBR.y2 for child in self.child_nodes])
        if self.parent_node and not self.parent_node.MBR.contain_rect(self.MBR):
            self.parent_node.update_MBR()
        pass

    # Get perimeter of an MBR formed by a list of data points
    @staticmethod
    def get_points_MBR_perimeter(points):
        x1 = min([point.x for point in points])
        x2 = max([point.x for point in points])
        y1 = min([point.y for point in points])
        y2 = max([point.y for point in points])
        return Rect(x1, y1, x2, y2).perimeter()

    @staticmethod
    def get_nodes_MBR_perimeter(nodes):
        x1 = min([node.MBR.x1 for node in nodes])
        x2 = max([node.MBR.x2 for node in nodes])
        y1 = min([node.MBR.y1 for node in nodes])
        y2 = max([node.MBR.y2 for node in nodes])
        return Rect(x1, y1, x2, y2).perimeter()


class RegionTree:
    def __init__(self, d,n):
        self.d = d
        self.n = n
        self.root = Node(self.d,self.n)

    def insert_point(self, point, cur_node=None):
        # init U as node
        # print("{} is leaf: {}".format(self.root, self.root.is_leaf()))
        if cur_node is None:
            cur_node = self.root

            # print("{} is leaf: {}".format(cur_node, cur_node.is_leaf()))
        # Insertion logic start
        if cur_node.is_leaf():
            cur_node.add_point(point)
            # handle overflow
            if cur_node.is_overflow():
                self.handle_overflow(cur_node)
        else:
            chosen_child = self.choose_best_child(cur_node, point)
            self.insert_point(point, cur_node=chosen_child)

    # Find a suitable one to expand:
    @staticmethod
    def choose_best_child(node, point):
        best_child = None
        best_perimeter = 0
        # Scan the child nodes
        for item in node.child_nodes:
            if node.child_nodes.index(item) == 0 or best_perimeter > item.perimeter_increase_with_point(point):
                best_child = item
                best_perimeter = item.perimeter_increase_with_point(point)
        return best_child

    # WIP
    def handle_overflow(self, node):
        node, new_node = self.split_leaf_node(node) if node.is_leaf() else self.split_internal_node(node)

        if self.root is node:
            self.root = Node(self.d, self.n)
            self.root.add_child_nodes([node, new_node])
        else:
            node.parent_node.add_child_node(new_node)
            if node.parent_node.is_overflow():
                self.handle_overflow(node.parent_node)

    # WIP
    def split_leaf_node(self, node):
        m = len(node.data_points)
        best_perimeter = -1
        best_set_1 = []
        best_set_2 = []
        # Run x axis
        all_point_sorted_by_x = sorted(node.data_points, key=lambda point: point.x)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_x[:i]
            list_point_2 = all_point_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        # Run y axis
        all_point_sorted_by_y = sorted(node.data_points, key=lambda point: point.y)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_y[:i]
            list_point_2 = all_point_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        node.data_points = best_set_1
        node.update_MBR()
        new_node = Node(self.d, self.n)
        new_node.add_points(best_set_2)
        return node, new_node

    # WIP
    def split_internal_node(self, node):
        m = len(node.child_nodes)
        best_perimeter = -1
        best_set_1 = []
        best_set_2 = []
        # Run x axis
        all_node_sorted_by_x = sorted(node.child_nodes, key=lambda child: child.MBR.x1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_x[:i]
            list_node_2 = all_node_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
                # Run y axis
        all_node_sorted_by_y = sorted(node.child_nodes, key=lambda child: child.MBR.y1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_y[:i]
            list_node_2 = all_node_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter == -1 or best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
        node.child_nodes = best_set_1
        node.update_MBR()
        new_node = Node(self.d, self.n)
        new_node.add_child_nodes(best_set_2)
        return node, new_node


# TASK 1

In [4]:
import pandas as pd
data = pd.read_csv('AllPOI Simplified.csv',header = None)
data

,0,1
0,116.303382,39.908620
1,116.802076,40.335921
2,116.807570,40.362750
3,116.839759,40.371410
4,117.105753,40.140551
...,...,...
182318,117.240095,40.615747
182319,117.111129,40.213956
182320,116.796412,40.562527
182321,116.459869,39.852378


In [32]:
def getHeight(node):
    if not node:
        return 0
    if not node.child_nodes:
        return 1
    return max([getHeight(i) for i in node.child_nodes]) + 1

In [48]:
d = 8
n = 64
tree = RegionTree(d,n)

print("n:",n)
print("d:",d)
for i in range(182323):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:",leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 64
d: 8
height: 6
leaf_num: 4308
non_leaf_num: 991


In [49]:
d = 8
n = 256
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(182323):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 256
d: 8
height: 6
leaf_num: 1064
non_leaf_num: 252


In [50]:
d = 32
n = 64
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(182323):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 64
d: 32
height: 4
leaf_num: 4264
non_leaf_num: 216


In [51]:
d = 32
n = 256
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(182323):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 256
d: 32
height: 4
leaf_num: 1056
non_leaf_num: 57


In [53]:
first_half_data = data.iloc[:int(len(data) * 0.5)]
first_half_data

,0,1
0,116.303382,39.908620
1,116.802076,40.335921
2,116.807570,40.362750
3,116.839759,40.371410
4,117.105753,40.140551
...,...,...
91156,116.791879,39.776471
91157,116.568320,39.863927
91158,116.491312,40.093498
91159,116.486482,40.093619


In [54]:
d = 8
n = 64
tree = RegionTree(d,n)

print("n:",n)
print("d:",d)
for i in range(len(first_half_data)):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:",leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 64
d: 8
height: 6
leaf_num: 2161
non_leaf_num: 497


In [55]:
d = 8
n = 256
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(len(first_half_data)):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 256
d: 8
height: 5
leaf_num: 537
non_leaf_num: 127


In [56]:
d = 32
n = 64
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(len(first_half_data)):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 64
d: 32
height: 4
leaf_num: 2118
non_leaf_num: 114


In [57]:
d = 32
n = 256
tree = RegionTree(d,n)

print("n:", n)
print("d:", d)
for i in range(len(first_half_data)):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))
print("height:", getHeight(tree.root))

leaf_num = 0
non_leaf_num = 0

def getLeafNumAndNonleafNum(node):
    global leaf_num
    global non_leaf_num
    if node.is_leaf():
        leaf_num = leaf_num + 1
    else:
        non_leaf_num = non_leaf_num + 1
    for i in range(len(node.child_nodes)):
        getLeafNumAndNonleafNum(node.child_nodes[i])
getLeafNumAndNonleafNum(tree.root)

print("leaf_num:", leaf_num)
print("non_leaf_num:", non_leaf_num)

n: 256
d: 32
height: 3
leaf_num: 535
non_leaf_num: 29


# TASK 2

In [5]:
def get_distance_ES(point_x, point_y):
    dist = float('inf')
    index = 0
    for i in range(182323):
        if data.iloc[i,0] == point_x and data.iloc[i,1] == point_y:
            continue
        else:
            tempD = math.sqrt(pow(point_x - data.iloc[i,0], 2) + pow(point_y - data.iloc[i,1], 2))
            if tempD < dist:
                index = i
                dist = tempD
    return index, dist 


In [6]:
tree = RegionTree(8,256)
for i in range(182323):
    tree.insert_point(Point(i, data.iloc[i,0],data.iloc[i,1]))

In [30]:
def get_four_dist_MBR(p, Rect):
    return min(math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y1,2)) , math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y2,2))
              ,math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y1,2)),math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y2,2)))

def get_min_dist(p, Rect):
    min_dict = 0
    if p.x >= Rect.x1 and p.x <= Rect.x2 and p.y >= Rect.y1 and p.y <= Rect.y2:
        return min_dict
    return min(get_four_dist_MBR(p,Rect), abs(p.y-Rect.y1), abs(p.y-Rect.y2), abs(p.y-Rect.x1),abs(p.y-Rect.x2))
    
def get_min_in_X(p, Rect):
    if abs(Rect.x1 - p.x) < abs(Rect.x2 - p.x):
        dist_inX = max(math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y1,2)),math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y2,2)))
    else:
        dist_inX = max(math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y1,2)),math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y2,2)))
    
    return dist_inX

def get_min_in_Y(p, Rect):
    if abs(Rect.y1 - p.y) < abs(Rect.y2 - p.y):
        dist_inY = max(math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y1,2)),math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y1,2)))
    else:
        dist_inY = max(math.sqrt(pow(p.x - Rect.x1,2) + pow(p.y - Rect.y2,2)),math.sqrt(pow(p.x - Rect.x2,2) + pow(p.y - Rect.y2,2)))
    
    return dist_inY

def get_min_in_dimension(p, Rect):
    
    dist_inX = get_min_in_X(p, Rect)
    
    dist_inY = get_min_in_Y(p, Rect)
    
    return min(dist_inX,dist_inY)


def RT_search(p, root):
    global min_in_dimension
    global min_dist
    global pruned1
    global pruned2
    global pruned3
    if root.is_leaf():
        point_count = 0
        for point in root.data_points:
            dist = math.sqrt(pow(p.x - point.x,2) + pow(p.y - point.y,2))
            point_count += 1
            if dist > min_in_dimension:
                pruned2 += 1
                continue
            if dist < min_dist:
                min_dist = dist
                near_point[0], near_point[1] = point.x, point.y
        return point_count
    total_count = 0
    for child in root.child_nodes:
        if child.is_leaf():
            total_count += RT_search(p, child)
            continue
        temp_min_dist = get_min_dist(p,child.MBR)
        temp_min_in_dimension = get_min_in_dimension(p,child.MBR)
        
        if temp_min_in_dimension < min_in_dimension:
            min_in_dimension = temp_min_in_dimension
        if temp_min_dist > min_in_dimension:
            pruned1 += 1
            continue
        if temp_min_dist > min_dist:
            pruned3 += 1
            continue
        total_count += RT_search(p, child)
    return total_count

In [31]:
import random
import time
left_most = min(data[0])
right_most = max(data[0])
up_most = max(data[1])
down_most = min(data[1])
flag = 1
ES_time_list = []
RT_time_list = []



for i in range(0, 30):
    point_x = round(random.uniform(left_most,right_most),6)
    
    point_y = round(random.uniform(down_most,up_most),6)
    
    print("exhaustive search:")
    start_t = time.perf_counter() 
    index, dist = get_distance_ES(point_x, point_y)
    end_t = time.perf_counter() 
    used_t = end_t - start_t
    ES_time_list.append(used_t)
    
    print("q point:", data.iloc[index][0],data.iloc[index][1])
    print("distance from p to q:",dist)
    print("time used:",used_t)

    
    print("R-tree search:")
    min_in_dimension = float('inf')
    min_dist = float('inf')
    pruned1 = 0
    pruned2 = 0
    pruned3 = 0
    near_point = [0,0]
    start_t = time.perf_counter() 
    total_count = RT_search(Point(i,point_x,point_y), tree.root)
    end_t = time.perf_counter() 
    used_t = end_t - start_t
    RT_time_list.append(used_t)
    print("time used:",used_t)
    print("number of search points",total_count)
    print("number of pruned MBR by rule1,2,3",pruned1,pruned2,pruned3)

exhaustive search:
q point: 117.418635 40.646328
distance from p to q: 0.03862610788055026
time used: 3.000000106112566e-07
R-tree search:
time used: 0.028616700000043238
number of search points 19437
number of pruned MBR by rule1,2,3 38 17694 17
exhaustive search:
q point: 117.418635 40.646328
distance from p to q: 0.03862610788055026
time used: 2.0000004496978363e-07
R-tree search:
time used: 0.028216400000019348
number of search points 20950
number of pruned MBR by rule1,2,3 2 20075 7
exhaustive search:
q point: 117.418635 40.646328
distance from p to q: 0.03862610788055026
time used: 1.999999312829459e-07
R-tree search:
time used: 0.01454480000006697
number of search points 10470
number of pruned MBR by rule1,2,3 37 7652 4
exhaustive search:
q point: 117.418635 40.646328
distance from p to q: 0.03862610788055026
time used: 2.0000004496978363e-07
R-tree search:
time used: 0.13684990000001562
number of search points 99308
number of pruned MBR by rule1,2,3 9 96280 9
exhaustive search:

In [157]:
import numpy as np

print("exhaustive search min time:", min(ES_time_list))
print("exhaustive search max time:", max(ES_time_list))
print("exhaustive search avg time:", np.mean(ES_time_list))


print("R-tree search min time:", min(RT_time_list))
print("R-tree search max time:", max(RT_time_list))
print("R-tree search avg time:", np.mean(RT_time_list))

exhaustive search min time: 11.402738299992052
exhaustive search max time: 13.532266899987007
exhaustive search avg time: 11.83551824333214
R-tree search min time: 0.01202999999804888
R-tree search max time: 0.2880405000032624
R-tree search avg time: 0.09866523333427418
